# ヒトTTN遺伝子のイントロンの5'、3'側末端の塩基の保存性の評価

**目的**
イントロン同士の配列を比較して、相同性を確認する<br>
相同性とスプライシングパターンの関連がないかを検証

In [ ]:
# オリジナルモジュールのインポート
from lib.gbkparse import Seq_count
from lib.geneinfo import gene_id

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import subprocess
import glob
import requests

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## 準備

In [ ]:
gid= str(gene_id('TTN').iloc[0,0])
gid

In [ ]:
# クラスのインスタンス化
gbk = Seq_count()

# gbkファイルの読み込み
# gbk.read_gbk('../data/gbk/human_ttn.gb')
gbk.get_gbk(gid)

## UCSCのTbale BrowserからダウンロードしたphastCons100wayのデータの読み込み

In [ ]:
# UCSCのデータベースからphastCons100wayの情報を取得
api_url = "https://api.genome.ucsc.edu/getData/track"
params = {
    "genome": "hg38",
    "track": "phastCons100way",  # 適切なトラック名を使用
    "chrom": f"{gbk.chrom}",  # 対象の染色体
    "start": f"{gbk.start}",  # 指定する領域の開始位置
    "end": f"{gbk.end}"  # 指定する領域の終了位置
}

response = requests.get(api_url, params=params)
if response.ok:
    cons_verts_data = response.json()
else:
    print("Error: Unable to fetch data")

In [ ]:
# プラス鎖の場合
if gbk.strand == 1:
    cons_df = pd.DataFrame(cons_verts_data[f'chr{gbk.chrom}']) # 保存性のJSONデータをデータフレームに変換
    cons_df = cons_df.loc[:,['start','value']] # 必要な情報のみを抽出
    cons_df.columns = ['pos','cons'] # カラム名を変更
    cons_df = cons_df.sort_values('pos') # 位置情報順にソート
    cons_df['pos'] = cons_df['pos'] - cons_df['pos'].min() # 遺伝子の5'側末端を0として位置情報を変換
    cons_df = cons_df.set_index('pos') # 位置情報をインデックスとして変換
    tmp = pd.DataFrame({'pos':[i for i in range(gbk.length)]}) # 上記情報は一部欠落があるので穴埋めようのデータフレームを用意
    cons_df = pd.merge(tmp, cons_df, left_index=True, right_index=True, how='outer') # 上記のtmpとcons_dfを外部結合
    cons_df = cons_df.set_index('pos', drop=True) # 'pos'情報をインデックスに

# マイナス鎖の場合
elif gbk.strand == 2:
    cons_df = pd.DataFrame(cons_verts_data[f'chr{gbk.chrom}']) # 保存性のJSONデータをデータフレームに変換
    cons_df = cons_df.loc[:,['start','value']] # 必要な情報のみを抽出
    cons_df.columns = ['pos','cons'] # カラム名を変更
    cons_df = cons_df.sort_values('pos') # 位置情報順にソート
    cons_df['pos'] = cons_df['pos'].max() - cons_df['pos'] # マイナス鎖であることを考慮しながら、遺伝子の5'側末端を0として位置情報を変換
    cons_df = cons_df.set_index('pos') # 位置情報をインデックスとして変換
    tmp = pd.DataFrame({'pos':[i for i in range(gbk.length)]}) # 上記情報は一部欠落があるので穴埋めようのデータフレームを用意
    cons_df = pd.merge(tmp, cons_df, left_index=True, right_index=True, how='outer') # 上記のtmpとcons_dfを外部結合
    cons_df = cons_df.set_index('pos', drop=True) # 'pos'情報をインデックスに
cons_df

## 各イントロンとエクソンの塩基ごとの保存性の平均値を求める

In [ ]:
# エクソンのごとの保存性の平均値を求める
exon_cons_ave = {}
for i in range(gbk.exon_num()):
    r = gbk.exon_list()[i]
    ave = round(cons_df.iloc[r[0]:r[1], 0].mean(),2)
    exon_cons_ave[i+1] = ave

# イントロンのごとの保存性の平均値を求める
intron_cons_ave = {}
for i in range(gbk.intron_num()):
    r = gbk.intron_list()[i]
    ave = round(cons_df.iloc[r[0]:r[1], 0].mean(),2)
    intron_cons_ave[i+1] = ave

In [ ]:
# 上記のデータの可視化
exon_hovertxt = 'Exon: %{x}<br>Conservation: %{y}<extra></extra>'
intron_hovertxt = 'Intron: %{x}<br>Conservation: %{y}<extra></extra>'
fig = make_subplots(rows=2, cols=1, subplot_titles=("各エクソンの塩基の保存性の平均", "各イントロンの塩基の保存性の平均"))
fig.add_trace(go.Bar(x=list(exon_cons_ave.keys()), y=list(exon_cons_ave.values()), hovertemplate=exon_hovertxt), row=1, col=1)
fig.add_trace(go.Bar(x=list(intron_cons_ave.keys()), y=list(intron_cons_ave.values()), hovertemplate=intron_hovertxt), row=2, col=1)
fig.update_yaxes(range=[0, 1], row=1, col=1)
fig.update_yaxes(range=[0, 1], row=2, col=1)
fig.update_layout(height=500, width=1200, title_text="塩基の保存性の平均値")
fig.show()

## 各イントロン末端からnum塩基の各塩基の保存性を可視化

In [ ]:
# 検討するイントロン末端からの塩基数
num = 80

# 5'側のイントロンn塩基の保存性の平均
left_intron_cons = {} 
for n, i in enumerate(gbk.intron_list()):
    left_intron_cons[n+1] = list(cons_df.iloc[i[0]:i[0]+num,0])
left_intron_cons_ave_df = pd.DataFrame.from_dict(left_intron_cons).T
left_intron_cons_ave_df.columns = [i for i in range(1, num+1)]
left_intron_cons_ave_df = left_intron_cons_ave_df.apply(lambda x: round(x,2))
# 5'側のイントロンn塩基の保存性の平均
right_intron_cons = {} 
for n, i in enumerate(gbk.intron_list()):
    right_intron_cons[n+1] = list(cons_df.iloc[i[1]-num:i[1],0])
right_intron_cons_ave_df = pd.DataFrame.from_dict(right_intron_cons).T
right_intron_cons_ave_df.columns = [-i for i in range(num, 0, -1)]
right_intron_cons_ave_df = right_intron_cons_ave_df.apply(lambda x: round(x,2))

In [ ]:
# 上記データの可視化
fig = make_subplots(rows=1, cols=2, subplot_titles=(f"各イントロンの5'側の{num}塩基の保存性", f"各イントロンの3'側の{num}塩基の保存性"))
left_hovertxt = 'Intron: %{y}<br>Base: %{x}<br>Conservation: %{z}<extra></extra>'
right_hovertxt = 'Intron: %{y}<br>Base: %{x}<br>Conservation: %{z}<extra></extra>'
fig.add_trace(go.Heatmap(x=left_intron_cons_ave_df.columns, y=left_intron_cons_ave_df.index, z=left_intron_cons_ave_df.values, colorscale='Viridis', hovertemplate=left_hovertxt), row=1, col=1)
fig.add_trace(go.Heatmap(x=right_intron_cons_ave_df.columns, y=right_intron_cons_ave_df.index, z=right_intron_cons_ave_df.values, colorscale='Viridis', hovertemplate=right_hovertxt), row=1, col=2)
fig.update_yaxes(autorange="reversed")
fig.update_layout(height=750, width=1200, title_text="サブプロットの例")
fig.show()